In [ ]:
import os
import subprocess
import psutil
import time
from IPython.display import display, HTML

def find_matlab_proxy_app_processes():
    processes = []
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        for process in proc.info['cmdline']:
            if 'matlab-proxy-app' in process:
                processes.append(proc.info)
    return processes

def find_next_open_port(
    *, host="0.0.0.0", start_port: int = 3000, end_port: int = 9999
):
    import socket

    for port in range(start_port, end_port + 1):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)  # Set a timeout for the connection attempt
        result = sock.connect_ex((host, port))
        if result != 0:  # 0 means the port is open
            sock.close()
            return port

    return None

port = find_next_open_port()

my_env = os.environ
my_env["MWI_APP_PORT"] = str(port)

start_msg = f"Starting matlab-proxy-app on port {str(port)}"
print(start_msg)
r = subprocess.Popen(["matlab-proxy-app"], env=my_env)

html_content = f"<h2 style='color:green;'>{start_msg}</h2>"

matlab_proxy_app_processes = find_matlab_proxy_app_processes()
html_content += f"<h2 style='color:blue;'>Found {len(matlab_proxy_app_processes)} matlab-proxy-app processes with process ids:</h2>"
for mp_process in matlab_proxy_app_processes:
    html_content += f"<h3 style='color:blue;'>{mp_process['pid']}</h3>"

display(HTML(html_content))
